In [3]:
from torchvision.transforms import transforms #библиотека для работы с изображениями
import torch

In [4]:
#open CV
import cv2

image = cv2.imread('pok.png')
image.shape

#возвращает np.array
#открывает картинку в BGR (обратно RGB)
#сначала ширина высота, потом каналы

(2206, 1938, 3)

In [5]:
from torchvision.io import read_image
#сначала количество каналов, а потом ширина - высота

image = read_image('pok.png')
image.shape


#возвращает tensor

torch.Size([4, 2206, 1938])

In [6]:
f_t = transforms.Lambda(lambda x: x*2) #Объект занесенный в функцию должен быть возведен в квадрат
a = torch.rand(4)
f_t(a)
#transforms.  -- посмотреть какие есть функции. Большинство функций работают с изображениями формата PIL (ToPILImage()) - поэтому надо менять форматы
#compose - п

tensor([0.3916, 1.8721, 1.8003, 0.0995])

In [7]:
f_t1 = transforms.Compose([f_t, f_t]) #объединяет функции в одну функцию, включая функции из transform.
f_t1(a)

tensor([0.7832, 3.7442, 3.6006, 0.1989])

In [8]:
#Если датасет слишком маленький, можно
#Изменять изображения - переворачивать, добавлять блюр, менять цветовую гамму
#2 Способ - взять натренированную модель. Взять слои которые натренированны на поиске низкоуровневых паттернов. И переобучаем модель на что-то новое

In [9]:
from torch.utils.data import Dataset
import pathlib 
#dataset - откуда забрать 1 объект
#dataloader - собирает батчи

In [10]:
class LatinDataset(Dataset):

    #в ините объявляем путь к датасету
    def __init__(self, path_dataset: pathlib.Path):
        self.path_dataset = path_dataset
        self.data_list = [x for x in self.path_dataset.glob('**/*') if x.is_file()] # **/* - паттерн значащий - любая папка внутри и не важно что после неё
        # то есть передаем в класс условие по которому ходить по датасету
        # ------------
        # Выделяем количество классов:
        self.data_class = list(set([x.parent for x in self.data_list])) #Используем множество т.к. там значения уникальны. Т.е. убираем повторяющиеся пути
        # Возвращаем в лист, т.к. в сете нет индексов, а они нам нужны
        # Для свертки важно лишь количество фильтров, а не размерность картинки.
        # Но для батчей все картинки должны быть одной размерности. Поэтому все картинки нужно подогнать к одному размеру.
        self.transform_func = transforms.Compose([
             transforms.ToPILImage(),
             transforms.Grayscale(), #т.к. мы работаем с чб картинками
             transforms.Resize((124, 124)),
             transforms.PILToTensor()
        ])

    def __len__(self): #всегда определять длину. Т.к. обращаемся к датасету он должен возвращать максимальное кол-во объектов в датасете
        return len(self.data_list)
    
    def __getitem__(self, index): #dataloader returns some index from datalist
        img_path = self.data_list[index] #path to the image
        #but path also returns label
        #label вектор размерности максимального размера класса, где единица указывает принадлежность к классу (см. one hot encoder)
        img_label = torch.zeros(len(self.data_class)) #возвращает тензор с нулевыми значениями с размерностью кол-ва классов
        img_label[self.data_class.index(img_path.parent)] = 1.0 #Находим индекс класса и заменяем ячейку под этим индексом на принадлежность к классу.
        #В нашем случае каждая папка это отдельный класс


        img_tensor = read_image(img_path)
        return self.transform_func(img_tensor), img_label
    

In [24]:
g = pathlib.Path(r'C:\Users\Joe\Desktop\python_yes\what_is_love\letters')
a = [x for x in g.glob('**/*') if x.is_file()]
set([x.parent for x in a])
#[a for a in g.iterdir()]

{WindowsPath('C:/Users/Joe/Desktop/python_yes/what_is_love/letters/A')}